In [ ]:
!pip install -qU langchain
!pip install -qU langchain-google-genai
!pip install -q sentence-transformers
!pip install -q pinecone
!pip install -q pypdf
!pip install -q opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
# testing gemini model through langchain integration
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

api_gemini = userdata.get('GEMINI')

test = ChatGoogleGenerativeAI(google_api_key=api_gemini, model="gemini-2.5-flash")
messages = [("system", "to every question you get, you just respond: 'I'm allergic to cats'"),
            ("user", "What do you think about Donald Trump?")]
response = test.invoke(messages)
print(response.content)

I'm allergic to cats


In [ ]:
# uploading files
from google.colab import files

pdf = files.upload()
image = files.upload()
video = files.upload()

Saving virtual_kaffe.pdf to virtual_kaffe.pdf


Saving virtual_kaffe_menu.jpg to virtual_kaffe_menu.jpg


Saving virtual_kaffe_video.mp4 to virtual_kaffe_video.mp4


In [ ]:
# processing data
from pypdf import PdfReader
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import cv2

api_gemini = userdata.get('GEMINI')

# text from the PDF
extracted = PdfReader("/content/virtual_kaffe.pdf")
text = extracted.pages[0].extract_text()
# print(text)

## using agentic chunking to chunk the text in semantically coherent chunks
chunking_model = ChatGoogleGenerativeAI(google_api_key=api_gemini, model='gemini-2.5-flash')
messages = [("system", """You're an AI Agent specialized in the pre-processing of data before being used to create embeddings.

                          Given the data you're handled, create chunks that semantically make sense, grouping pieces of the text
                          by ideas.

                          Output the data in this format: n_chunk_here * n+1_chunk_here * n+2_chunk_here * ...

                          IMPORTANT:
                          - Make sure to use asterisks as delimitators between chunks
                          - Make sure not to include text formatting characters like '\n' or bullet points"""),
            ("user", text)]

response = chunking_model.invoke(messages)
text_list = response.content.split(sep=" * ")
# print(text_list)

# frames from the video
video = cv2.VideoCapture('/content/virtual_kaffe_video.mp4')
frame_counter = 1

while True:
  ret, frame = video.read()

  if not ret:
    break

  match frame_counter:
    case 1:
      filename = f"frame_{frame_counter}.jpg"
      cv2.imwrite(filename, frame)
      print(f"Frame {frame_counter} saved.")
    case 30:
      filename = f"frame_{frame_counter}.jpg"
      cv2.imwrite(filename, frame)
      print(f"Frame {frame_counter} saved.")
    case 60:
      filename = f"frame_{frame_counter}.jpg"
      cv2.imwrite(filename, frame)
      print(f"Frame {frame_counter} saved.")
    case 90:
      filename = f"frame_{frame_counter}.jpg"
      cv2.imwrite(filename, frame)
      print(f"Frame {frame_counter} saved.")
    case 120:
      filename = f"frame_{frame_counter}.jpg"
      cv2.imwrite(filename, frame)
      print(f"Frame {frame_counter} saved.")



  frame_counter += 1



Frame 1 saved.
Frame 30 saved.
Frame 60 saved.
Frame 90 saved.
Frame 120 saved.


In [ ]:
# setting up models for embeddings
from sentence_transformers import SentenceTransformer
from PIL import Image

model = SentenceTransformer('clip-ViT-B-32') # we use clip because it understands both images and text

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
img_emb = model.encode(Image.open('/content/virtual_kaffe_menu.jpg'))
text_emb = model.encode(text_list)
video
print(img_emb.shape, text_emb.shape)

(512,) (7, 512)
